# Rearrange Argentina SOC data 

In [1]:
import numpy as np
import rasterio
import os
from google.cloud import storage

## SOC concentration dataset

**Input data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/

**Data description:**

The name structure of the files is `SOC_YEAR_qQUANTILE_dDEPTH.tif`:

- YEAR: 1982-2017
- QUANTILE: 0.05,0.5,0.95 percentiles
- DEPTH:
    - 2.5 --> for the interval 0-5cm
    - 10 --> for the interval 5-15cm
    - 22.5 --> for the interval 15-30cm
    - 45 --> for the interval 30-60cm
    - 80 --> for the interval 60-100cm
    - 150 --> for the interval 100-200cm

All data are in geotiff in projection EPSG:4326.
The data are SOC * 10 in order to use integers and therefore save space.
To obtain the real SOC values the data need to be divided by 10 (or the units need to be changed).

**Output data location:**

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_concentration_q5

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_concentration_q50

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_concentration_q95

**Move objects**

In [71]:
def move_blob(bucket_name, privatekey_path, source_file_prefix, destination_path):
    """Move files in a bucket."""
    bucket_name = 'vizz-data-transfer'
    privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"
    
    storage_client = storage.Client.from_service_account_json(privatekey_path)
    bucket = storage_client.get_bucket(bucket_name)
    
    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=source_file_prefix)
    
    for blob in blobs:
        new_name = blob.name.split('/')[-1]
        new_name =  destination_path + new_name
        print(blob.name)
        bucket.copy_blob(blob, destination_bucket = bucket, new_name = new_name)
        #bucket.delete_blob(blob.name)  

In [70]:
bucket_name = 'vizz-data-transfer'
privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"

move_blob(bucket_name, privatekey_path, source_file_prefix='SOC_maps/SOC_', destination_path='SOC_maps/SOC_concentration/')

SOC_maps/Feb19_cstocks_1982_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1982_030_ll.tif
SOC_maps/Feb19_cstocks_1983_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1983_030_ll.tif
SOC_maps/Feb19_cstocks_1984_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1984_030_ll.tif
SOC_maps/Feb19_cstocks_1985_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1985_030_ll.tif
SOC_maps/Feb19_cstocks_1986_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1986_030_ll.tif
SOC_maps/Feb19_cstocks_1987_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1987_030_ll.tif
SOC_maps/Feb19_cstocks_1988_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1988_030_ll.tif
SOC_maps/Feb19_cstocks_1989_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1989_030_ll.tif
SOC_maps/Feb19_cstocks_1990_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1990_030_ll.tif
SOC_maps/Feb19_cstocks_1991_030_ll.tif
SOC_maps/SOC_concentration/Feb19_cstocks_1991_030_ll.tif
SOC_maps/Feb19_cstocks_1992_030_ll.tif
S

### Rearrange `.tif` files

We save the different depths as bands of the same image

In [2]:
def upload_blob(bucket_name, source_file_name, destination_blob_name, privatekey_path):
    """Uploads a file to the bucket."""
    storage_client = storage.Client.from_service_account_json(privatekey_path)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [29]:
years = (np.arange(36)+1982).astype(np.str)
depths = ['2.5', '10', '22.5', '45', '80', '150']
quantiles = ['0.05', '0.5', '0.95']
base_url = 'https://storage.googleapis.com/vizz-data-transfer/SOC_maps/'

bucket_name = 'vizz-data-transfer'
privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"

for quantile in quantiles:
    print('Quantile', quantile)
    # Read metadata of first file
    with rasterio.open(base_url+'SOC_'+str(years[0])+'_q'+quantile+'_d'+depths[0]+'.tif') as src0:
        profile = src0.profile
    
    # Update metadata to reflect the number of bands
    profile.update(count = len(depths))

    for i, year in enumerate(years):
        
        with rasterio.open('./data/SOC_'+year+'_q'+quantile+'.tif', 'w', **profile) as dst:
            for id, depth in enumerate(depths, start=1):
                with rasterio.open(base_url+'SOC_'+year+'_q'+quantile+'_d'+depth+'.tif') as src1:
                    dst.write_band(id, src1.read(1))
                
        upload_blob(bucket_name, './data/SOC_'+year+'_q'+quantile+'.tif', 'SOC_maps/SOC_'+year+'_q'+quantile+'.tif', privatekey_path)

        os.remove('./data/SOC_'+year+'_q'+quantile+'.tif')

Quantile 0.05


/anaconda3/lib/python3.6/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


File ./data/SOC_1982_q0.05.tif uploaded to SOC_maps/SOC_1982_q0.05.tif.
File ./data/SOC_1983_q0.05.tif uploaded to SOC_maps/SOC_1983_q0.05.tif.
File ./data/SOC_1984_q0.05.tif uploaded to SOC_maps/SOC_1984_q0.05.tif.
File ./data/SOC_1985_q0.05.tif uploaded to SOC_maps/SOC_1985_q0.05.tif.
File ./data/SOC_1986_q0.05.tif uploaded to SOC_maps/SOC_1986_q0.05.tif.
File ./data/SOC_1987_q0.05.tif uploaded to SOC_maps/SOC_1987_q0.05.tif.
File ./data/SOC_1988_q0.05.tif uploaded to SOC_maps/SOC_1988_q0.05.tif.
File ./data/SOC_1989_q0.05.tif uploaded to SOC_maps/SOC_1989_q0.05.tif.
File ./data/SOC_1990_q0.05.tif uploaded to SOC_maps/SOC_1990_q0.05.tif.
File ./data/SOC_1991_q0.05.tif uploaded to SOC_maps/SOC_1991_q0.05.tif.
File ./data/SOC_1992_q0.05.tif uploaded to SOC_maps/SOC_1992_q0.05.tif.
File ./data/SOC_1993_q0.05.tif uploaded to SOC_maps/SOC_1993_q0.05.tif.
File ./data/SOC_1994_q0.05.tif uploaded to SOC_maps/SOC_1994_q0.05.tif.
File ./data/SOC_1995_q0.05.tif uploaded to SOC_maps/SOC_1995_q0.

### Create a GEE ImageCollection 

In [30]:
years = (np.arange(36)+1982).astype(np.str)
quantiles = ['0.05', '0.95'] #['0.05', '0.5', '0.95']
quant_per = ['5', '95'] #['5', '50', '95']

for i, quantile in enumerate(quantiles):
    print('Quantile', quantile)
    quant_name = quant_per[i]
    # We create a collection asset in Earth Engine to use as a write target.
    !earthengine create collection users/iker/SOC_maps/SOC_concentration_q{quant_name}

    # Next, we use the upload image command to send individual images to the collection.
    for year in years:
        year = str(year)
    
        !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id users/iker/SOC_maps/SOC_concentration_q{quant_name}/SOC_{year} gs://vizz-data-transfer/SOC_maps/SOC_{year}_q{quantile}.tif

Quantile 0.05
Started upload task with ID: JJGQDHT4JCJB55YUAG5TMQXJ
Started upload task with ID: H2JYVQRWUSUOOAY5ZZUYLLWC
Started upload task with ID: KTM7SRQZQSVWGR6UF6HLZULR
Started upload task with ID: IMS76ABJKBSC5M3M5GS2ESHM
Started upload task with ID: IBOJLZG2T2S7WGFPILAL6KDZ
Started upload task with ID: Q5KQHSTZSOUMFRNUT47PHBDJ
Started upload task with ID: MI6AFXYFICKK3AALZFNCACJA
Started upload task with ID: GKUKR4D7VJYAFVEZC6A3GXXP
Started upload task with ID: WM45BA5UDNQTTIENNLHQPVZZ
Started upload task with ID: R7JOKDWBKAKFCIZIOJVHY5NZ
Started upload task with ID: S26V2KH7IQU3MGATPTVGGR73
Started upload task with ID: 75A4FWKKVV2OAMDS46V3I3YM
Started upload task with ID: MMYKALXNCQFGDF6Y7VXWS4HO
Started upload task with ID: 2BYMRZQ4K4F6GB5HID7Y3UXJ
Started upload task with ID: N2VBZEFEMQ53W7AZCZAH62XU
Started upload task with ID: IGWAUSEINNMPIVP6Q5QDZDIE
Started upload task with ID: EBTLXH77VEFZHL2Z3X5QRQQZ
Started upload task with ID: NPKW4BHO4JSUT66H5K5E7GNX
Started upload

## SOC stocks dataset

**Data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/

**Data description:**

The name structure of the files is `Feb19_cstocks_YEAR_030_ll.tif``:
- YEAR: 1982-2017
- The stocks were calculated in the 0 to 30 cm interval. 

**Output data location:**

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_stock_0_30

**Move objects**

In [72]:
bucket_name = 'vizz-data-transfer'
privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"

move_blob(bucket_name, privatekey_path, source_file_prefix='SOC_maps/Feb19', destination_path='SOC_maps/SOC_stock/')

SOC_maps/Feb19_cstocks_1982_030_ll.tif
SOC_maps/Feb19_cstocks_1983_030_ll.tif
SOC_maps/Feb19_cstocks_1984_030_ll.tif
SOC_maps/Feb19_cstocks_1985_030_ll.tif
SOC_maps/Feb19_cstocks_1986_030_ll.tif
SOC_maps/Feb19_cstocks_1987_030_ll.tif
SOC_maps/Feb19_cstocks_1988_030_ll.tif
SOC_maps/Feb19_cstocks_1989_030_ll.tif
SOC_maps/Feb19_cstocks_1990_030_ll.tif
SOC_maps/Feb19_cstocks_1991_030_ll.tif
SOC_maps/Feb19_cstocks_1992_030_ll.tif
SOC_maps/Feb19_cstocks_1993_030_ll.tif
SOC_maps/Feb19_cstocks_1994_030_ll.tif
SOC_maps/Feb19_cstocks_1995_030_ll.tif
SOC_maps/Feb19_cstocks_1996_030_ll.tif
SOC_maps/Feb19_cstocks_1997_030_ll.tif
SOC_maps/Feb19_cstocks_1998_030_ll.tif
SOC_maps/Feb19_cstocks_1999_030_ll.tif
SOC_maps/Feb19_cstocks_2000_030_ll.tif
SOC_maps/Feb19_cstocks_2001_030_ll.tif
SOC_maps/Feb19_cstocks_2002_030_ll.tif
SOC_maps/Feb19_cstocks_2003_030_ll.tif
SOC_maps/Feb19_cstocks_2004_030_ll.tif
SOC_maps/Feb19_cstocks_2005_030_ll.tif
SOC_maps/Feb19_cstocks_2006_030_ll.tif
SOC_maps/Feb19_cstocks_20

In [6]:
years = (np.arange(36)+1982).astype(np.str)

# We create a collection asset in Earth Engine to use as a write target.
!earthengine create collection users/iker/SOC_maps/SOC_stock_0_30

# Next, we use the upload image command to send individual images to the collection.
for year in years:
    !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id users/iker/SOC_maps/SOC_stock_0_30/SOC_stock_{year} gs://vizz-data-transfer/SOC_maps/SOC_stock/Feb19_cstocks_{year}_030_ll.tif

Asset users/iker/SOC_maps/SOC_stock_0_30 already exists
Started upload task with ID: FINN27EPXHUYNNZKYOI2NI77
Started upload task with ID: PC7HWO4C24FGDDTNXRWKCGOT
Started upload task with ID: 5CR244KVESGTV2M47VG4IRNH
Started upload task with ID: H4YBQZS4QP433WZRTO74ZYUU
Started upload task with ID: SI6QBZO6WXET5D4JVQSTHGRO
Started upload task with ID: ODCONT676OKB3RM7Y343DFRP
Started upload task with ID: GPS6GWC5BPNXJW46VPHBYQ5D
Started upload task with ID: WR3DE22SJ54BNM6LO76G4GDG
Started upload task with ID: IBFF7DTOAOZBLWLDWBR63NT4
Started upload task with ID: KYLZJAR562RMMW7IG4IVDF6T
Started upload task with ID: UH4EIJIH2PNA3SSICEENDV2U
Started upload task with ID: 3BMVYZ66TXK6X5TEJD3JSV6I
Started upload task with ID: DZ4LF7YNC4GJTSLRQDA2GN2X
Started upload task with ID: AUZ2MKFR5LAKLIT46VGCXFKI
Started upload task with ID: RQTKYVB57HZXPQ4EGBPTZWQQ
Started upload task with ID: LJ7GQOKE2BWO7QPVKBPHCAAA
Started upload task with ID: QQQCW6HQHZQ44H4HBYRWGTN4
Started upload task with I

## Soil carbon debt of 12,000 years of human land use
**Data location:**
- Current stock https://github.com/whrc/Soil-Carbon-Debt/blob/master/SOCS/SOCS_0_200cm_year_2010AD_10km.tif
- Historic stock https://github.com/whrc/Soil-Carbon-Debt/blob/master/SOCS/SOCS_0_200cm_year_NoLU_10km.tif

**Output data location:**

- Current stock https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOCS_0_200cm_year_2010AD_10km
- Historic stock https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOCS_0_200cm_year_NoLU_10km